In [1]:
import os
import re
import pandas as pd
import numpy as np
import collections
import itertools
import subprocess
from tqdm import tqdm as tqdm
from concurrent.futures import ThreadPoolExecutor

# Utils

In [2]:
def cartesianproduct(ranges):
    sets = set(itertools.product(*ranges))
    return sets


# def calculate_score(plot_txt, preds=['X2'], targets=['O2']):
#     with open(plot_txt, 'r') as fin:
#         lines = fin.readlines()
#         title = lines[0]
#         tokens = title.split()
        
#         lines = [line.split() for line in lines[1:]]
        
#         df = pd.DataFrame(data=np.array(lines), dtype=np.float32, columns=tokens)
    
#     score = 0.0
#     for index in range(len(targets)):
#         score += ((df[f"{targets[index]}"]-df[f'{preds[index]}'])**2).mean()
    
#     return score


# ./pil.sh control_v2/original 10000 r1_cat "X1 X2 O1 O2" r1_cat
def execute(rn, 
            param_ks,
            param_vs,
            time='1000', 
            labels="X1 X2 O1 O2",
            preds=['X2'],
            targets=['O2']):
    try:
        for index, param_k in enumerate(param_ks):
            rn = rn.replace(f'-{param_k}-', str(param_vs[index]))
        p = subprocess.Popen(f'''./pil_stdout.sh {time} \"{labels}\" \"{rn}\"''',
                             shell=True, 
                             stdout=subprocess.PIPE, 
                             stderr=subprocess.STDOUT)
        lines = p.stdout.readlines()
        lines = [line.strip().decode(encoding='UTF-8', errors='ignore') for line in lines]
        header = lines[0].split()
        lines = [line.split() for line in lines[1:]]
#         print(lines[:4])
        df = pd.DataFrame(data=np.array(lines), dtype=np.float32, columns=header)
        score = 0.0
        for index in range(len(targets)):
            score += ((df[f"{targets[index]}"]-df[f'{preds[index]}'])**2).mean()
        retval = p.wait()
        return score
    except:
        return 10000.0


def find_optimal_param_values(rn, 
                              params, 
                              ranges,
                              time=1000,
                              labels="X1 X2 O1 O2",
                              preds=["X2"],
                              targets=["O2"]):
    
    sets = list(cartesianproduct(ranges))
    sets.sort()
    
    best_score = 10000.0 
    max_paramset = None
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        for set, score in tqdm(zip(sets, executor.map(lambda s: execute(rn,
                                                    params,
                                                    s,
                                                    time=time,
                                                    labels=labels,
                                                    preds=preds,
                                                    targets=targets
                                                    ), sets))):
            
            if best_score < score: 
                best_score = score
                max_paramset = set
    
    print(best_score)
    print(max_paramset)
        
            
    
    
    
#     for paramset in tqdm(sets):
        
#         # Substitute the values of all the params
#         rn_this = rn
#         for index, param in enumerate(params):
#             rn_this = rn_this.replace(f'-{param}-', str(paramset[index]))
            
#         new_enum_file = os.path.join(root_dir, f'test_{basefile}')   
#         with open(new_enum_file, 'w') as fout:
#             fout.write(rn_this)
        
#         new_enum_file_base = os.path.basename(new_enum_file)
#         new_enum_file_base=new_enum_file_base.replace('_enum.pil', '')

#         score = execute(
#             root_dir=root_dir,
#             time=time,
#             base=new_enum_file_base,
#             labels=labels,
#             preds=['X2'], 
#             targets=['O2']
#         )
#         if score < best_score: # Score is the cost vector. 
#             best_score = score
#             max_paramset = paramset
        
#         print(f'{paramset} | score: {score}')
#         print(f'{max_paramset} | best_score: {best_score}')
#         print('-------------------------------')
        

# Cat

In [3]:
params = [
    'ReactX1',
    'BackX1',
    'HelperX1',
    'ProduceX1X2',
    'HelperX1X2'
]

ranges = [
    np.logspace(1, 3, 6),
    np.logspace(1, 3, 6),
    np.logspace(1, 3, 6),
    np.logspace(1, 3, 6),
    np.logspace(1, 3, 6)
]

In [4]:
sets = list(cartesianproduct(ranges))
sets.sort()

In [5]:
rn = f'''
# File generated by peppercorn-v1.1.1

# Domains (27) 
length fB = 7
length fO1 = 7
length fO2 = 7
length fX1 = 7
length fX2 = 7
length hO1 = 15
length hO2 = 15
length hX1 = 15
length hX2 = 15
length mB = 15
length mO1 = 15
length mO2 = 15
length mX1 = 15
length mX2 = 15
length sO1 = 7
length sO2 = 7
length sX1 = 7
length sX2 = 7

# Resting complexes (14) 
BackX1 = fB sX1 mX1 @initial -BackX1- nM
ReactX1 = hX1 sX2( mB( + fB( sX1( mX1( + fX1* ) ) ) ) ) @initial -ReactX1- nM
ReactInt = sX1( mX1( fX1( hX1 + ) ) ) fB* mB*( sX2*( + hX1 ) )
R1_28 = sX1( mX1( fX1( hX1 + ) ) ) fB*( mB*( sX2*( + ) ) )
HelperX1X2 = fX2 hX2 fX1 @initial -HelperX1X2- nM
R1_20 = hX1( sX2( mB + ) ) fX1*( hX2*( fX2*( + ) ) )
FluxX1B = hX1 sX2 mB
ProduceInt = hX1( sX2( mB + ) ) fX1* hX2*( fX2*( + sX2 mX2 ) )
O1 = sO1 mO1 fO1 hO1 @initial 2 nM
O2 = sO2 mO2 fO2 hO2 @initial 1e-05 nM
X1 = sX1 mX1 fX1 hX1 @initial 2 nM
ProduceX1X2 = sX2 mX2 fX2( hX2( + sX1 mX1 fX1( hX1( + sX2* ) ) ) ) @initial -ProduceX1X2- nM
HelperX1 = sX2 mB fB @initial -HelperX1- nM
X2 = sX2 mX2 fX2 hX2 @initial 1e-05 nM

# Resting macrostates (14) 
macrostate BackX1 = [BackX1]
macrostate ReactX1 = [ReactX1]
macrostate ReactInt = [ReactInt]
macrostate R1_28 = [R1_28]
macrostate HelperX1X2 = [HelperX1X2]
macrostate R1_20 = [R1_20]
macrostate FluxX1B = [FluxX1B]
macrostate ProduceInt = [ProduceInt]
macrostate O1 = [O1]
macrostate O2 = [O2]
macrostate X1 = [X1]
macrostate ProduceX1X2 = [ProduceX1X2]
macrostate HelperX1 = [HelperX1]
macrostate X2 = [X2]

# Condensed reactions (6) 
reaction [condensed    = 0.00104661 /nM/s ] BackX1 + ReactInt -> ReactX1 + X1
reaction [condensed    = 0.00104661 /nM/s ] ReactX1 + X1 -> BackX1 + ReactInt
reaction [condensed    =  0.0020865 /nM/s ] ReactInt + HelperX1 -> R1_28 + FluxX1B
reaction [condensed    =  0.0020865 /nM/s ] HelperX1X2 + ProduceInt -> R1_20 + X2
reaction [condensed    = 0.00104769 /nM/s ] FluxX1B + ProduceX1X2 -> ProduceInt + X1
reaction [condensed    = 0.00104769 /nM/s ] ProduceInt + X1 -> FluxX1B + ProduceX1X2
reaction [condensed    = 0.0008 /s] O1 -> O1 + O2
'''

In [6]:
# calculate_score('control_v2/original/plots/test_r1_cat')
# execute(rn, params, sets[0])

In [ ]:
find_optimal_param_values(rn, # Don't substitute it.
                          params, 
                          ranges,
                          time=1000,
                          labels="X1 X2 O1 O2",
                          preds=["X2"],
                          targets=["O2"])

In [1]:
import os

In [4]:
os.path.dirname('control_v2/original/plots/test_r1_cat.pil')

'control_v2/original/plots'